In [15]:
import requests
import json
import pandas as pd
from pandas import json_normalize

#공공데이터 포털에서 가져온 api 키, https://www.data.go.kr/
mykey = "qtdamt6TFAWE1Ay9G5om8dMA2ecfdfLHrttgbGw7boA0pWESuMORtcCflm0HkcRsTuobjsPnq%2BVFj49N98qkSA%3D%3D"

#한국 공휴일 csv 생성하고 생성 여부를 반환하는 함수
def korean_holiday_csv_maker(year):
    url = (
            "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear="
            + str(year)
            + "&ServiceKey="
            + str(mykey)
        )
    response = requests.get(url)

    #정상적으로 정보를 받아오면 받아온 공휴일 정보를 csv로 저장
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob["response"]["body"]["items"]["item"]
        dataframe = json_normalize(holidays_data)
        dataframe.to_csv('{}_Korean_holiday.csv'.format(year))
        print('생성 성공!')
        return True
    print('생성 실패!')
    return False

korean_holiday_csv_maker(2023)
korean_holiday_csv_maker(2024)

생성 성공!
생성 성공!


True

In [18]:
holiday_data_23 = pd.read_csv('data/extra/2023_Korean_holiday.csv', encoding = 'utf-8')
print(holiday_data_23.head())

   Unnamed: 0  dateKind dateName isHoliday   locdate  seq
0           0         1     1월1일         Y  20230101    1
1           1         1       설날         Y  20230121    1
2           2         1       설날         Y  20230122    1
3           3         1       설날         Y  20230123    1
4           4         1    대체공휴일         Y  20230124    1


In [19]:
holiday_data_24 = pd.read_csv('data/extra/2024_Korean_holiday.csv', encoding = 'utf-8')
print(holiday_data_24.head())

   Unnamed: 0  dateKind   dateName isHoliday   locdate  seq
0           0         1       1월1일         Y  20240101    1
1           1         1         설날         Y  20240209    1
2           2         1         설날         Y  20240210    1
3           3         1         설날         Y  20240211    1
4           4         1  대체공휴일(설날)         Y  20240212    1


In [20]:
holiday_2324 = pd.DataFrame()

# 2023년 데이터 추가
holiday_2324['year'] = holiday_data_23['locdate'].astype(str).str[:4]  # 2023년 데이터
holiday_2324['month'] = holiday_data_23['locdate'].astype(str).str[4:6]
holiday_2324['day'] = holiday_data_23['locdate'].astype(str).str[6:8]

# 2024년 데이터 추가
holiday_2024 = pd.DataFrame({
    'year': holiday_data_24['locdate'].astype(str).str[:4],  # 2024년 데이터
    'month': holiday_data_24['locdate'].astype(str).str[4:6],
    'day': holiday_data_24['locdate'].astype(str).str[6:8]
})

# holiday_2324에 2024년 데이터 추가
holiday_2324 = pd.concat([holiday_2324, holiday_2024], ignore_index=True)

# 결과 확인
print(holiday_2324.head())  # 처음 5행 출력
print(holiday_2324.tail())  # 마지막 5행 출력

   year month day
0  2023    01  01
1  2023    01  21
2  2023    01  22
3  2023    01  23
4  2023    01  24
    year month day
32  2024    09  18
33  2024    10  01
34  2024    10  03
35  2024    10  09
36  2024    12  25


In [5]:
train_fri_data = pd.read_csv('data/processed/train_금요일data1.csv')
train_weekend_data = pd.read_csv('data/processed/train_주말data1.csv')
train_weekday_data = pd.read_csv('data/processed/train_주중data1.csv')

/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_5539/2421352419.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  train_fri_data = pd.read_csv('data/processed/train_금요일data1.csv')
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_5539/2421352419.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  train_weekend_data = pd.read_csv('data/processed/train_주말data1.csv')
/var/folders/tg/ntvm9wc94h937xqtzftck9xc0000gn/T/ipykernel_5539/2421352419.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  train_weekday_data = pd.read_csv('data/processed/train_주중data1.csv')


In [6]:
# 'holiday' 열 추가 (초기값 0)
train_fri_data['holiday'] = 0
train_weekend_data['holiday'] = 0
train_weekday_data['holiday'] = 0

In [7]:
# holiday_2324 DataFrame의 'year', 'month', 'day'를 기준으로 공휴일 여부 확인
for index, row in train_fri_data.iterrows():
    year = str(row['year'])   # train_fri_data의 'year' 열에서 연도 가져오기
    month = str(row['month']).zfill(2)  # train_fri_data의 'month' 열에서 월 가져오기
    day = str(row['day']).zfill(2)      # train_fri_data의 'day' 열에서 일 가져오기

    # holiday_2324에 해당 날짜가 존재하는지 확인
    if ((holiday_2324['year'] == year) &
        (holiday_2324['month'] == month) &
        (holiday_2324['day'] == day)).any():
        train_fri_data.at[index, 'holiday'] = 1  # 공휴일이면 1 저장

# 결과 확인
print(train_fri_data.head())  # 'holiday' 열이 추가된 train_fri_data의 처음 5행 출력

     차량이름                 승차시간    승차요일       승차X좌표      승차Y좌표  \
0  iarnsg  2023-04-07 00:00:18  Friday  127.350577  36.305774   
1  eiihxy  2023-04-07 00:03:53  Friday  127.372759  36.327395   
2  zazemd  2023-04-07 00:05:15  Friday  127.381975  36.347150   
3  hzdykr  2023-04-07 00:05:31  Friday  127.391151  36.423884   
4  iarnsg  2023-04-07 00:10:41  Friday  127.337384  36.330387   

                  하차시간       하차X좌표      하차Y좌표  승차거리(m) 할증여부  ...  \
0  2023-04-07 00:08:18  127.340533  36.330138  4625.00    1  ...   
1  2023-04-07 00:10:53  127.385410  36.341310  2124.84    1  ...   
2  2023-04-07 00:11:54  127.372210  36.321143  3833.45    1  ...   
3  2023-04-07 00:12:31  127.383478  36.440981  2697.93    1  ...   
4  2023-04-07 00:16:54  127.332609  36.339160  1683.63    1  ...   

              주행시간 주행시간_초  hour  year  month  day  기온(°C)  강수량(mm)   group  \
0  0 days 00:08:00  480.0     0  2023      4    7     7.1      1.1  Friday   
1  0 days 00:07:00  420.0     0  2023      4

In [8]:
# holiday_2324 DataFrame의 'year', 'month', 'day'를 기준으로 공휴일 여부 확인
for index, row in train_weekend_data.iterrows():
    year = str(row['year'])   # train_weekend_data의 'year' 열에서 연도 가져오기
    month = str(row['month']).zfill(2)  # train_weekend_data의 'month' 열에서 월 가져오기
    day = str(row['day']).zfill(2)      # train_weekend_data의 'day' 열에서 일 가져오기

    # holiday_2324에 해당 날짜가 존재하는지 확인
    if ((holiday_2324['year'] == year) &
        (holiday_2324['month'] == month) &
        (holiday_2324['day'] == day)).any():
        train_weekend_data.at[index, 'holiday'] = 1  # 공휴일이면 1 저장

# 결과 확인
print(train_weekend_data.head())  # 'holiday' 열이 추가된 train_fri_data의 처음 5행 출력

     차량이름                 승차시간      승차요일       승차X좌표      승차Y좌표  \
0  bfllzp  2023-04-01 00:04:45  Saturday  127.336672  36.307763   
1  clspqc  2023-04-01 00:06:20  Saturday  127.411853  36.336721   
2  dmpnpp  2023-04-01 00:05:10  Saturday  127.434345  36.328179   
3  hzdykr  2023-04-01 00:10:31  Saturday  127.339601  36.348446   
4  eiihxy  2023-04-01 00:04:28  Saturday  127.389238  36.349626   

                  하차시간       하차X좌표      하차Y좌표  승차거리(m) 할증여부  ...  \
0  2023-04-01 00:11:28  127.345800  36.323435   2531.0    1  ...   
1  2023-04-01 00:12:18  127.420696  36.346687   2086.0    1  ...   
2  2023-04-01 00:13:34  127.461915  36.330595   3682.0    1  ...   
3  2023-04-01 00:15:13  127.353202  36.361700   2594.0    1  ...   
4  2023-04-01 00:18:24  127.345296  36.341787   5259.0    1  ...   

              주행시간 주행시간_초  hour  year  month  day  기온(°C)  강수량(mm)    group  \
0  0 days 00:06:43  403.0     0  2023      4    1    12.1      0.0  Weekend   
1  0 days 00:05:58  358.0     

In [9]:
# holiday_2324 DataFrame의 'year', 'month', 'day'를 기준으로 공휴일 여부 확인
for index, row in train_weekday_data.iterrows():
    year = str(row['year'])   # train_weekday_data의 'year' 열에서 연도 가져오기
    month = str(row['month']).zfill(2)  # train_weekday_data의 'month' 열에서 월 가져오기
    day = str(row['day']).zfill(2)      # train_weekday_data의 'day' 열에서 일 가져오기

    # holiday_2324에 해당 날짜가 존재하는지 확인
    if ((holiday_2324['year'] == year) &
        (holiday_2324['month'] == month) &
        (holiday_2324['day'] == day)).any():
        train_weekday_data.at[index, 'holiday'] = 1  # 공휴일이면 1 저장

# 결과 확인
print(train_weekday_data.head())  # 'holiday' 열이 추가된 train_fri_data의 처음 5행 출력

     차량이름                 승차시간    승차요일       승차X좌표      승차Y좌표  \
0  clspqc  2023-04-03 00:00:16  Monday  127.392879  36.338629   
1  clspqc  2023-04-03 00:06:15  Monday  127.375892  36.348033   
2  cytplg  2023-04-03 00:01:19  Monday  127.350632  36.361790   
3  gxpwil  2023-04-03 00:00:27  Monday  127.377370  36.351706   
4  clspqc  2023-04-03 00:14:09  Monday  127.382204  36.334894   

                  하차시간       하차X좌표      하차Y좌표  승차거리(m) 할증여부  ...  \
0  2023-04-03 00:04:20  127.373209  36.347611   2372.0    1  ...   
1  2023-04-03 00:08:45  127.378054  36.336707   1449.0    1  ...   
2  2023-04-03 00:13:18  127.390382  36.348084   5642.0    1  ...   
3  2023-04-03 00:13:45  127.359768  36.374444   4728.0    1  ...   
4  2023-04-03 00:19:20  127.397827  36.332249   2222.0    1  ...   

              주행시간 주행시간_초  hour  year  month  day  기온(°C)  강수량(mm)     group  \
0  0 days 00:04:04  244.0     0  2023      4    3    14.8      0.0  Weekdays   
1  0 days 00:02:30  150.0     0  2023   

In [10]:
train_weekday_data.head()

,차량이름,승차시간,승차요일,승차X좌표,승차Y좌표,하차시간,하차X좌표,하차Y좌표,승차거리(m),할증여부,...,주행시간,주행시간_초,hour,year,month,day,기온(°C),강수량(mm),group,holiday
0,clspqc,2023-04-03 00:00:16,Monday,127.392879,36.338629,2023-04-03 00:04:20,127.373209,36.347611,2372.0,1,...,0 days 00:04:04,244.0,0,2023,4,3,14.8,0.0,Weekdays,0
1,clspqc,2023-04-03 00:06:15,Monday,127.375892,36.348033,2023-04-03 00:08:45,127.378054,36.336707,1449.0,1,...,0 days 00:02:30,150.0,0,2023,4,3,14.8,0.0,Weekdays,0
2,cytplg,2023-04-03 00:01:19,Monday,127.350632,36.361790,2023-04-03 00:13:18,127.390382,36.348084,5642.0,1,...,0 days 00:11:59,719.0,0,2023,4,3,14.8,0.0,Weekdays,0
3,gxpwil,2023-04-03 00:00:27,Monday,127.377370,36.351706,2023-04-03 00:13:45,127.359768,36.374444,4728.0,1,...,0 days 00:13:18,798.0,0,2023,4,3,14.8,0.0,Weekdays,0
4,clspqc,2023-04-03 00:14:09,Monday,127.382204,36.334894,2023-04-03 00:19:20,127.397827,36.332249,2222.0,1,...,0 days 00:05:11,311.0,0,2023,4,3,14.8,0.0,Weekdays,0


In [12]:
train_weekday_data.to_csv('data/processed/train_주중data2.csv', index=False)

In [13]:
train_fri_data.to_csv('data/processed/train_금요일data2.csv', index=False)

In [14]:
train_weekend_data.to_csv('data/processed/train_주말data2.csv', index=False)